In [14]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy
import re

from gensim.models import CoherenceModel, LdaMulticore, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
from gensim.test.utils import datapath
import pyLDAvis.gensim

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Import all_reviews_clean_sentiment

In [3]:
# bring in all_reviews df with cleaned reviews
all_reviews_clean_sentiment = pd.read_pickle('./Data/all_reviews_clean_sentiment.pkl')
all_reviews_clean_sentiment.head()

,date_of_stay,hotel_response,month_of_stay,nationality,review,source,trip_type,year_of_stay,review_clean,TextBlob_Polarity,vader_score
0,2010-01,NaN,January,Malaysia,Brunei” overall am satisfied as the hotel is ...,Agoda,Couple,2010,brunei overall be satisfy a the hotel be very ...,0.275694,0.9708
1,2010-01,NaN,January,Indonesia,great week-end at Mandarin Meritus” It's been ...,Agoda,Group,2010,great week end at mandarin meritus it s be alm...,0.256061,0.9030
2,2010-01,NaN,January,Malaysia,Favourite Hotel !” Mandarin Meritus has always...,Agoda,Business traveler,2010,favourite hotel mandarin meritus have always b...,0.466077,0.9642
3,2010-01,NaN,January,Malaysia,nice” very nice hotel. nice food and great roo...,Agoda,Couple,2010,nice very nice hotel nice food and great room ...,0.162812,0.4507
4,2010-01,NaN,January,Indonesia,"meritus orchard” Very nice hotel,location real...",Agoda,Couple,2010,meritus orchard very nice hotel location reall...,0.408462,0.9777


## Use spacy to prep data for topic modeling

In [4]:
nlp = spacy.load("en")

In [5]:
my_stop_words = [u'hotel', u'orchard', u'road', u'shop', u'singapore', u'mandarin', u'meritus', u'location', u'locate',\
                 u'good', u'great', u'excellent', u'business', u'trip', u'day', u'night', u'star', u'view',\
                 u'trip', u'center', u'centre', u'station', u'city', u'rd', u'lot', u'say', u'\'s', u'Mr',\
                 u'be', u'said', u'says', u'saying']
for stopword in my_stop_words:
    lexeme = nlp.vocab[stopword]
    lexeme.is_stop = True

In [6]:
all_reviews_clean_sentiment['review'] = all_reviews_clean_sentiment['review'].str.replace(".", ' ')

In [7]:
def clean_up(text): 
    text_out = []
    doc= nlp(text)
    for token in doc:
        if token.is_stop == False and token.like_num == False and token.is_punct == False and token.pos_ == 'NOUN':
            text_out.append(token.lemma_)
    return text_out

In [8]:
all_reviews_clean_sentiment['review_spacyclean'] = all_reviews_clean_sentiment['review']\
.apply(lambda x: clean_up(x))

In [9]:
dictionary = Dictionary(all_reviews_clean_sentiment['review_spacyclean'])

In [10]:
corpus = [dictionary.doc2bow(text) for text in all_reviews_clean_sentiment['review_spacyclean']]

In [40]:
# hdpmodel = HdpModel(corpus=corpus, id2word=dictionary)
# [print(topic,'\n') for topic in hdpmodel.show_topics()]

In [25]:
ldamodel = LdaMulticore(corpus=corpus, num_topics=4, id2word=dictionary, passes=500, workers=5)

In [26]:
[print(topic,'\n') for topic in ldamodel.show_topics()]

(0, '0.068*"room" + 0.045*"check" + 0.043*"time" + 0.037*"staff" + 0.036*"service" + 0.016*"guest" + 0.015*"hour" + 0.013*"luggage" + 0.012*"stay" + 0.011*"reception"') 

(1, '0.144*"room" + 0.035*"bed" + 0.019*"bathroom" + 0.019*"floor" + 0.015*"shower" + 0.013*"stay" + 0.013*"water" + 0.011*"service" + 0.011*"tower" + 0.010*"wing"') 

(2, '0.067*"breakfast" + 0.051*"staff" + 0.051*"service" + 0.046*"room" + 0.032*"food" + 0.031*"stay" + 0.018*"club" + 0.017*"time" + 0.016*"buffet" + 0.014*"restaurant"') 

(3, '0.111*"shopping" + 0.048*"room" + 0.037*"mall" + 0.029*"place" + 0.028*"staff" + 0.026*"food" + 0.026*"restaurant" + 0.026*"area" + 0.019*"heart" + 0.018*"service"') 



[None, None, None, None]

In [27]:
ldacoherence = CoherenceModel(model=ldamodel, corpus=corpus, dictionary=dictionary, coherence='u_mass')
print(ldacoherence.get_coherence())

-2.455665549625013


## VIZ

In [28]:
pyLDAvis.enable_notebook()
vis_data = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)

In [29]:
vis_data

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.123514 -0.143034       1        1  27.403582
1      0.157975  0.169743       2        1  25.658331
2     -0.047416 -0.082506       3        1  25.433784
3     -0.234073  0.055797       4        1  21.504305, topic_info=     Category          Freq         Term         Total  loglift  logprob
4     Default   5094.000000     shopping   5094.000000  30.0000  30.0000
196   Default   3606.000000    breakfast   3606.000000  29.0000  29.0000
45    Default   2637.000000        check   2637.000000  28.0000  28.0000
80    Default   1712.000000         mall   1712.000000  27.0000  27.0000
56    Default   1897.000000          bed   1897.000000  26.0000  26.0000
23    Default   2917.000000         food   2917.000000  25.0000  25.0000
17    Default  16530.000000         room  16530.000000  24.0000  24.0000
25    Default   1881.000000        place   1881.000000  23.0000  23.0000
172   Default   1966.000000   restaurant   1966.000000  22.0000  22.0000
30    Default   1034.000000     bathroom   1034.000000  21.0000  21.0000
140   Default    956.000000         club    956.000000  20.0000  20.0000
637   Default    883.000000       buffet    883.000000  19.0000  19.0000
69    Default    989.000000        heart    989.000000  18.0000  18.0000
134   Default    816.000000       shower    816.000000  17.0000  17.0000
74    Default   6579.000000        staff   6579.000000  16.0000  16.0000
100   Default    660.000000         shop    660.000000  15.0000  15.0000
489   Default    933.000000         hour    933.000000  14.0000  14.0000
62    Default    744.000000      luggage    744.000000  13.0000  13.0000
105   Default    624.000000     distance    624.000000  12.0000  12.0000
651   Default    683.000000       lounge    683.000000  11.0000  11.0000
44    Default   1841.000000         area   1841.000000  10.0000  10.0000
346   Default    684.000000        water    684.000000   9.0000   9.0000
284   Default    750.000000       middle    750.000000   8.0000   8.0000
3     Default   6293.000000      service   6293.000000   7.0000   7.0000
7     Default    586.000000        value    586.000000   6.0000   6.0000
14    Default    851.000000       access    851.000000   5.0000   5.0000
37    Default    885.000000         pool    885.000000   4.0000   4.0000
104   Default    550.000000         wing    550.000000   3.0000   3.0000
227   Default    565.000000    concierge    565.000000   2.0000   2.0000
5     Default    988.000000         taxi    988.000000   1.0000   1.0000
...       ...           ...          ...           ...      ...      ...
586    Topic4     95.458405       subway     96.205101   1.5291  -6.1718
413    Topic4     92.163139  takashimaya     92.903755   1.5289  -6.2069
977    Topic4     89.058304      walking     89.799652   1.5286  -6.2412
409    Topic4     87.299973        metro     88.045486   1.5284  -6.2611
5228   Topic4     92.102615   smartphone     92.894470   1.5284  -6.2076
383    Topic4     77.713341     activity     78.500771   1.5268  -6.3774
834    Topic4     71.887207       eatery     72.643776   1.5264  -6.4554
284    Topic4    673.670166       middle    750.937805   1.4283  -4.2177
69     Topic4    869.509094        heart    989.662842   1.4075  -3.9625
79     Topic4    470.253387     location    576.808899   1.3327  -4.5772
25     Topic4   1308.681030        place   1881.718994   1.1738  -3.5537
14     Topic4    613.503235       access    851.870361   1.2087  -4.3113
44     Topic4   1172.339844         area   1841.269897   1.0855  -3.6637
172    Topic4   1195.505249   restaurant   1966.500732   1.0392  -3.6441
194    Topic4    218.216354      tourist    265.019684   1.3426  -5.3450
23     Topic4   1204.734863         food   2917.833252   0.6523  -3.6365
752    Topic4    257.516174          lot    350.616028   1.2283  -5.1794
390    Topic4    556.294800       family   1268.200684  

In [30]:
pyLDAvis.save_html(vis_data, './lda_topic_viz.html')

## save model

In [ ]:
# # load
# lda = LdaModel.load(temp_file)

In [31]:
ldamodel.save('./Model/lda_4topics_500.model')

# assign topic to each review

In [32]:
corpus_lda = ldamodel[corpus]

lda_topic = []

for doc in corpus_lda:
    lda_topic.append(doc)

In [33]:
lda_topic_frame = pd.DataFrame(lda_topic, columns=['topic_0', 'topic_1', 'topic_2', 'topic_3'])
lda_topic_frame.head(3)

,topic_0,topic_1,topic_2,topic_3
0,"(0, 0.38113126)","(1, 0.025298346)","(2, 0.35928825)","(3, 0.23428214)"
1,"(0, 0.90150136)","(1, 0.032604907)","(2, 0.03301195)","(3, 0.032881808)"
2,"(0, 0.032682512)","(1, 0.036005903)","(2, 0.032777984)","(3, 0.89853364)"


In [34]:
topic0_prob = pd.DataFrame(lda_topic_frame['topic_0'].tolist(), index=lda_topic_frame.index).iloc[:,1].rename(columns = {"1": "topic0_probability"})
topic1_prob = pd.DataFrame(lda_topic_frame['topic_1'].tolist(), index=lda_topic_frame.index).iloc[:,1].rename(columns = {"1": "topic1_probability"})
topic2_prob = pd.DataFrame(lda_topic_frame['topic_2'].tolist(), index=lda_topic_frame.index).iloc[:,1].rename(columns = {"1": "topic2_probability"})
topic3_prob = pd.DataFrame(lda_topic_frame['topic_3'].tolist(), index=lda_topic_frame.index).iloc[:,1].rename(columns = {"1": "topic3_probability"})

In [35]:
all_reviews_final = pd.concat([all_reviews_clean_sentiment, topic0_prob, topic1_prob, topic2_prob, topic3_prob], axis=1)

In [36]:
all_reviews_final['CustSvc/Checkin'] = np.where(all_reviews_final[0] >= 0.25, 1, 0)
all_reviews_final['Room'] = np.where(all_reviews_final[1] >= 0.25, 1, 0)
all_reviews_final['F&B/Breakfast'] = np.where(all_reviews_final[2] >= 0.25, 1, 0)
all_reviews_final['Gallery/Shopping'] = np.where(all_reviews_final[3] >= 0.25, 1, 0)

In [37]:
# export to csv
all_reviews_final.to_csv('./Data/all_reviews_final.csv')

In [38]:
all_reviews_final.head()

,date_of_stay,hotel_response,month_of_stay,nationality,review,source,trip_type,year_of_stay,review_clean,TextBlob_Polarity,vader_score,review_spacyclean,0,1,2,3,CustSvc/Checkin,Room,F&B/Breakfast,Gallery/Shopping
0,2010-01,NaN,January,Malaysia,Brunei” overall am satisfied as the hotel is ...,Agoda,Couple,2010,brunei overall be satisfy a the hotel be very ...,0.275694,0.9708,"[service, ppl, time, shopping, service, people...",0.381131,0.025298,0.359288,0.234282,1,0,1,0
1,2010-01,NaN,January,Indonesia,great week-end at Mandarin Meritus” It's been ...,Agoda,Group,2010,great week end at mandarin meritus it s be alm...,0.256061,0.9030,"[week, end, year, face, point, hotel, time]",0.901501,0.032605,0.033012,0.032882,1,0,0,0
2,2010-01,NaN,January,Malaysia,Favourite Hotel !” Mandarin Meritus has always...,Agoda,Business traveler,2010,favourite hotel mandarin meritus have always b...,0.466077,0.9642,"[pick, thousand, shopping, access, room, decor...",0.032683,0.036006,0.032778,0.898534,0,0,0,1
3,2010-01,NaN,January,Malaysia,nice” very nice hotel nice food and great roo...,Agoda,Couple,2010,nice very nice hotel nice food and great room ...,0.162812,0.4507,"[food, room, service, delicacy, distant, shopp...",0.017106,0.501469,0.017598,0.463827,0,1,0,1
4,2010-01,NaN,January,Indonesia,"meritus orchard” Very nice hotel,location real...",Agoda,Couple,2010,meritus orchard very nice hotel location reall...,0.408462,0.9777,"[shopping, condition, bedroom, bathroom, swimm...",0.013625,0.481790,0.167047,0.337538,0,1,0,1
